In [1]:
import sympy as sym
import sympy.physics.mechanics as me
from sympy.abc import t as time
import pandas as pd
import seaborn as sns

import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

import sys, os

sys.path.insert(1, os.path.join(sys.path[0], '../..'))
import custom_plot_objects as cpo

import ModelFramework as mf
import ModelFramework.Elements as ele
import ModelFramework.ExternalForces as ef
import FwtModels.RectWing as rw
import FwtModels.RollRig as RollRig

from matplotlib.lines import Line2D

me.mechanics_printing()

In [68]:
half = sym.Rational(1,2)
fwt_panels = 1
main_panels = 20
panels = main_panels + 2*fwt_panels

# create parameters
p = mf.ModelParameters.DynamicModel(3)

#Geometry Parameters
p.c = mf.ModelSymbol(value = 0.15,string = 'c') # chord of wing
p.s = mf.ModelSymbol(value = 1,string = 'S')    # span of wing
p.s_w = mf.ModelSymbol(value = 0.72,string = 'S_w')    # span of wing
p.A = mf.ModelSymbol(value = 0.72,string = 'A')    # span of wing

# Mass Parameter
p.m_w = mf.ModelSymbol(value = 0.22, string = 'm_w') # mass per unit span of inner wing
p.m_t = mf.ModelSymbol(value = 0.038,string = 'm_t') # mass of each FWT

p.I_xx_f = mf.ModelSymbol (value = 0.1, string = 'I_xxf')
p.I_yy_w = mf.ModelSymbol (value = 0.1, string = 'I_yyw')


# Attitude Parmas
p.rho = mf.ModelSymbol(value = 1.225,string = 'rho')                 # density
p.V = mf.ModelSymbol(value = 10,string = 'V')                        # velocity
p.g  = mf.ModelSymbol(value = 9.81,string = 'g')                     # gravity
p.alpha_r = mf.ModelSymbol(value = np.deg2rad(3),string = 'alpha_r') # root AoA
p.alpha_c = mf.ModelSymbol(value = 0, string = 'alpha_c')            # FWT camber

p.c_m_alpha = mf.ModelSymbol(value = 0, string = 'c_ma')            # FWT camber
p.c_m_q = mf.ModelSymbol(value = 0, string = 'c_mq')            # FWT camber
p.c_m_eta = mf.ModelSymbol(value = 0, string = 'c_m\eta')            # FWT camber
p.eta = mf.ModelSymbol(value = 0, string = 'eta')            # FWT camber

p.alpha_0 = mf.ModelSymbol(value = 0,string = 'alpha_0')             # FWT alpha
p.alphadot_0 = mf.ModelSymbol(value = 0,string = 'alphadot_0')       # FWT alphadot
p.alpha_1 = mf.ModelSymbol(value = 0,string = 'alpha_1')             # FWT alpha
p.alphadot_1 = mf.ModelSymbol(value = 0,string = 'alphadot_1')       # FWT alphadot
p.alpha_2 = mf.ModelSymbol(value = 0,string = 'alpha_2')             # FWT alpha
p.alphadot_2 = mf.ModelSymbol(value = 0,string = 'alphadot_2')       # FWT alphadot

p.c_d_max = mf.ModelSymbol(value = 1,string='C_Dmax')
p.a_0 = mf.ModelSymbol(value = 2*np.pi, string = 'a_0')   # C_L at the root
p.a_1 = mf.ModelSymbol(value = 2*np.pi, string = 'a_1')   # C_L at the root
p.beta = mf.ModelSymbol(value = 2*np.pi, string = 'beta' ) # Aileron Angle

# fwt params
p.Lambda = mf.ModelSymbol(value = np.deg2rad(10),string = 'Lambda') # flare angle
p.y_f = sym.Symbol('y_f')# spanwise location
p.y_i = sym.Symbol('y_i') # spanwise location

#Gust Velocity
p.w_g = mf.ModelSymbol(value = 0,string='w_g')

#CLosed loop gains for roll control
p.p = mf.ModelSymbol(value = 1,string='p')

# Time constnat for torque
p.T = mf.ModelSymbol(value = 1,string='T')

## Numeric Model Constants
p.a = mf.ModelMatrix(value =[np.pi*2]*panels,symbols=sym.symbols(f'a:{panels}')) # The stationary point

p.y_w = mf.ModelSymbol(value = 0,string = 'y_w') # y offset of wing CoM
p.z_w = mf.ModelSymbol(value = 0,string = 'z_w') # z offset of wing CoM
p.l_f = mf.ModelSymbol(value = 0,string = 'l_f') # moment arm of FWT

#integration constant
p.y_n = mf.ModelSymbol(value = 0,string = 'y_n') # integration constant


## FWTD Structural Parameters
p.y_0 = sym.Symbol('y_0')
p.x_0 = sym.Symbol('x_0')

## Numeric Model Constants
p.fp = mf.ModelMatrix(value =[0]*p.qs*2,symbols=sym.symbols(f'qtilde:{p.qs*2}')) # The stationary point

In [93]:
# Generate Referecne Frame
wing_frame = mf.HomogenousTransform().R_y(p.q[2])
rhs_fwt_frame = wing_frame.Translate(0,p.s_w,0).R_x(p.q[1])  # RHS Fwt Frame
lhs_fwt_frame = wing_frame.Translate(0,-p.s_w,0).R_x(-p.q[0])  # LHS Fwt Frame

#Generate Mass Matrices
M_w = ele.MassMatrix(p.m_w)
I_w = ele.MassMatrix(0,I_yy = p.I_yy_w)
fwt_w = ele.MassMatrix(p.m_t*(p.s-p.s_w),I_xx = sym.Rational(1,12)*p.m_t*(p.s-p.s_w)**3)

# Generate Rigid Elements
Wing_point_mass = ele.RigidElement(wing_frame,M_w,gravityPotential=True)
wing_inertia = ele.RigidElement(wing_frame,I_w)
rhs_fwt_mass = ele.RigidElement(rhs_fwt_frame.Translate(0,half*(p.s-p.s_w),0),fwt_w,gravityPotential=True)
lhs_fwt_mass = ele.RigidElement(lhs_fwt_frame.Translate(0,-half*(p.s-p.s_w),0),fwt_w,gravityPotential=True)

In [149]:
# Main Wing Aero Forces 
wing_AeroForces = ef.AeroForce.PerUnitSpan(p,wing_frame.Translate(0,p.y_i,0),p.a_0,
                               alphadot = 0,
                               M_thetadot = 0,
                               e = 0,
                               w_g = 0,
                               rootAlpha = p.q[2],
                               alpha_zero = p.alpha_c,
                               stall_angle = 0,
                               c = p.c,
                               c_d_max = 0,
                               linear = True).integrate((p.y_i,-p.s_w,p.s_w))

In [150]:
# split Main Wing into segments
# forces = []
# for i in range(main_panels):
#     seg_width = p.s_w/main_panels
#     yi = -p.s_w/2 + seg_width/2 + i*seg_width
#     forces.append(wing_AeroForces.subs({p.y_i:yi,p.a_0:p.a[fwt_panels+i]})*seg_width)
# Q = sym.Matrix([0]*p.qs)
# for f in forces:
#     Q += f.Q()
# wing_AeroForces = ef.ExternalForce(Q)

In [151]:
# Left FWT Aero Forces
lhs_fwt_AeroForces = ef.AeroForce.PerUnitSpan(p,lhs_fwt_frame.Translate(0,-p.y_f,0),p.a_0,
                               alphadot = 0,
                               M_thetadot = 0,
                               e = 0,
                               w_g = 0,
                               rootAlpha = p.q[2] - p.q[0]*sym.sin(p.Lambda),
                               alpha_zero = p.alpha_c,
                               stall_angle = 0,
                               c = p.c,
                               c_d_max = 0,
                               linear = True).integrate((p.y_f,0,(p.s-p.s_w)))

In [152]:
# Right FWT Aero Forces 
rhs_fwt_AeroForces = ef.AeroForce.PerUnitSpan(p,rhs_fwt_frame.Translate(0,p.y_f,0),p.a_0,
                               alphadot = 0,
                               M_thetadot = 0,
                               e = 0,
                               w_g = 0,
                               rootAlpha = p.q[2] - p.q[1]*sym.sin(p.Lambda),
                               alpha_zero = p.alpha_c,
                               stall_angle = 0,
                               c = p.c,
                               c_d_max = p.c_d_max,
                               linear = True).integrate((p.y_f,0,(p.s-p.s_w)))

In [153]:
rhs_fwt_AeroForces.Q()

⎡                                            0                                
⎢                                                                             
⎢                    3                 ⎛ 2               2                    
⎢  V⋅a₀⋅c⋅ρ⋅(S - S_w) ⋅q₁̇            2 ⎜V ⋅a₀⋅α_c⋅c⋅ρ   V ⋅a₀⋅c⋅ρ⋅q₁⋅sin(Λ)  
⎢- ────────────────────── + (S - S_w) ⋅⎜───────────── - ─────────────────── + 
⎢            6                         ⎝      4                  4            
⎢                                                                             
⎣                                            0                                

             ⎤
             ⎥
 2          ⎞⎥
 V ⋅a₀⋅c⋅ρ⋅q₂⎟⎥
────────────⎟⎥
     4      ⎠⎥
             ⎥
             ⎦

In [154]:
# split FTW's into segments
# forces = []
# for i in range(fwt_panels):
#     seg_width = (p.s-p.s_w)/fwt_panels
#     yi = seg_width/2 + i*seg_width
#     forces.append(lhs_fwt_AeroForces.subs({p.y_f:yi,p.a_0:p.a[fwt_panels-(i+1)]})*seg_width)
#     forces.append(rhs_fwt_AeroForces.subs({p.y_f:yi,p.a_0:p.a[fwt_panels+main_panels+i]})*seg_width)
# Q = sym.Matrix([0]*p.qs)
# for f in forces:
#     Q += f.Q()
# fwt_AeroForces = ef.ExternalForce(Q)



In [155]:
#Tail plane force
_Q = [0]*p.qs
_Q[2] = half*p.rho*p.V**2*p.A*(p.c_m_alpha*(p.alpha_c+p.q[2])+p.c_m_q*p.qd[2]+p.c_m_eta*p.eta)

tail_force = ef.ExternalForce(sym.Matrix(_Q))

sym.Matrix(_Q)

⎡                      0                      ⎤
⎢                                             ⎥
⎢                      0                      ⎥
⎢                                             ⎥
⎢   2                                         ⎥
⎢A⋅V ⋅ρ⋅(c_m\eta⋅η + cₘₐ⋅(α_c + q₂) + c_mq⋅q₂̇)⎥
⎢─────────────────────────────────────────────⎥
⎣                      2                      ⎦

In [156]:
Forcing = ef.CustomForce(None) 

CompositeForce = ef.CompositeForce([lhs_fwt_AeroForces,rhs_fwt_AeroForces,wing_AeroForces,Forcing,tail_force])
# create instance of the model
sm = mf.SymbolicModel.FromElementsAndForces(p,[Wing_point_mass,wing_inertia,rhs_fwt_mass,lhs_fwt_mass],ExtForces = CompositeForce)

In [157]:
sym.factor(sym.simplify(sm.M))

⎡   ⎛ 3      2              2      3⎞                                         
⎢mₜ⋅⎝S  - 3⋅S ⋅S_w + 3⋅S⋅S_w  - S_w ⎠                                         
⎢────────────────────────────────────                   0                     
⎢                 3                                                           
⎢                                                                             
⎢                                         ⎛ 3      2              2      3⎞   
⎢                                      mₜ⋅⎝S  - 3⋅S ⋅S_w + 3⋅S⋅S_w  - S_w ⎠   
⎢                 0                    ────────────────────────────────────   
⎢                                                       3                     
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                 0                                 

In [158]:
sm.M

⎡ 3                                 3                                         
⎢S ⋅mₜ    2               2      S_w ⋅mₜ                                      
⎢───── - S ⋅S_w⋅mₜ + S⋅S_w ⋅mₜ - ───────                     0                
⎢  3                                3                                         
⎢                                                                             
⎢                                          3                                 3
⎢                                         S ⋅mₜ    2               2      S_w 
⎢                   0                     ───── - S ⋅S_w⋅mₜ + S⋅S_w ⋅mₜ - ────
⎢                                           3                                3
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                   0                               

In [159]:
lin_sm = sm.linearise(p).subs({fp:p.fp.value[i]for i,fp in enumerate(p.fp)})

In [160]:
sym.simplify(lin_sm.M)

⎡   ⎛ 3      2              2      3⎞                                         
⎢mₜ⋅⎝S  - 3⋅S ⋅S_w + 3⋅S⋅S_w  - S_w ⎠                                         
⎢────────────────────────────────────                   0                     
⎢                 3                                                           
⎢                                                                             
⎢                                         ⎛ 3      2              2      3⎞   
⎢                                      mₜ⋅⎝S  - 3⋅S ⋅S_w + 3⋅S⋅S_w  - S_w ⎠   
⎢                 0                    ────────────────────────────────────   
⎢                                                       3                     
⎢                                                                             
⎣                 0                                     0                    I

    ⎤
    ⎥
 0  ⎥
    ⎥
    ⎥
    ⎥
    ⎥
 0  ⎥
    ⎥
    ⎥
_yyw⎦

In [161]:
p.qd

⎡q₀̇⎤
⎢  ⎥
⎢q₁̇⎥
⎢  ⎥
⎣q₂̇⎦

In [162]:
f = -(lin_sm.ExtForces.Q() - lin_sm.f)
C = f.jacobian(p.qd)
K = sym.simplify(f-C*p.qd).jacobian(p.q)
Q = sym.simplify(f-C*p.qd-K*p.q)
q = sym.Matrix([p.alpha_c,p.eta])
A = Q.jacobian(q)
B = Q-A*q

In [163]:
C

⎡                  3                                    ⎤
⎢V⋅a₀⋅c⋅ρ⋅(S - S_w)                                     ⎥
⎢───────────────────           0                 0      ⎥
⎢         6                                             ⎥
⎢                                                       ⎥
⎢                                       3               ⎥
⎢                     V⋅a₀⋅c⋅ρ⋅(S - S_w)                ⎥
⎢         0           ───────────────────        0      ⎥
⎢                              6                        ⎥
⎢                                                       ⎥
⎢                                              2        ⎥
⎢                                          -A⋅V ⋅c_mq⋅ρ ⎥
⎢         0                    0           ─────────────⎥
⎣                                                2      ⎦

In [164]:
K

⎡ 2                 2                                        2                
⎢V ⋅a₀⋅c⋅ρ⋅(S - S_w) ⋅sin(Λ)                               -V ⋅a₀⋅c⋅ρ⋅(S - S_w
⎢───────────────────────────               0               ───────────────────
⎢             4                                                      4        
⎢                                                                             
⎢                              2                 2           2                
⎢                             V ⋅a₀⋅c⋅ρ⋅(S - S_w) ⋅sin(Λ)  -V ⋅a₀⋅c⋅ρ⋅(S - S_w
⎢             0               ───────────────────────────  ───────────────────
⎢                                          4                         4        
⎢                                                                             
⎢                                                                   2         
⎢                                                               -A⋅V ⋅cₘₐ⋅ρ   
⎢             0                            0        

In [165]:
-A

⎡ 2                 2                ⎤
⎢V ⋅a₀⋅c⋅ρ⋅(S - S_w)                 ⎥
⎢────────────────────        0       ⎥
⎢         4                          ⎥
⎢                                    ⎥
⎢ 2                 2                ⎥
⎢V ⋅a₀⋅c⋅ρ⋅(S - S_w)                 ⎥
⎢────────────────────        0       ⎥
⎢         4                          ⎥
⎢                                    ⎥
⎢        2                2          ⎥
⎢     A⋅V ⋅cₘₐ⋅ρ       A⋅V ⋅c_m\eta⋅ρ⎥
⎢     ──────────       ──────────────⎥
⎣         2                  2       ⎦

In [166]:
q

⎡α_c⎤
⎢   ⎥
⎣ η ⎦

In [128]:
sym.simplify(-B)

⎡     ⎛ 2                2⎞⎤
⎢g⋅mₜ⋅⎝S  - 2⋅S⋅S_w + S_w ⎠⎥
⎢──────────────────────────⎥
⎢            2             ⎥
⎢                          ⎥
⎢     ⎛ 2                2⎞⎥
⎢g⋅mₜ⋅⎝S  - 2⋅S⋅S_w + S_w ⎠⎥
⎢──────────────────────────⎥
⎢            2             ⎥
⎢                          ⎥
⎣            0             ⎦

In [ ]:
sm.to_file('RollRigModel.py')

In [11]:
sm.M

⎡                                                                             
⎢                       2                                                     
⎢2⋅I_xxf + I_xxw + 2⋅l_f ⋅m_f - l_f⋅m_f⋅s⋅σ⋅cos(q₁) - l_f⋅m_f⋅s⋅σ⋅cos(q₂) + l_
⎢                                                                             
⎢                                                                             
⎢                                                                 2       l_f⋅
⎢                                                     -I_xxf - l_f ⋅m_f + ────
⎢                                                                             
⎢                                                                             
⎢                                                                 2       l_f⋅
⎢                                                      I_xxf + l_f ⋅m_f - ────
⎣                                                                             

                                           2  2    

In [11]:
rhs_fwt_mass.CalcKE(p)